#####Libraries
***

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import csv
from scipy.stats import norm
import statsmodels.api as sm
from patsy import dmatrices #a Python library for describing statistical models and building Design Matrices using R-like formulas.
import matplotlib.pyplot as plt
%matplotlib inline


In [36]:
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#####Load Dataset
***

In [37]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]

In [38]:
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-01-01','2022-04-01')['Adj Close'].index)

[*********************100%***********************]  1 of 1 completed


In [39]:
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [40]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,95.901833,75.405952,127.535355,111.874298,85.464371,181.170578,159.331497,146.124390,147.571686,114.787949,...,68.825935,231.919998,217.761978,37.217041,114.390381,340.295959,228.901291,214.385712,200.256943,37.317036
2021-01-05,96.893509,75.304222,127.573990,112.089272,88.150208,180.111160,160.925507,145.701553,147.892303,115.400581,...,69.492577,234.910004,215.967239,37.830185,114.488083,335.719727,234.802002,211.186096,199.358719,39.115448
2021-01-06,96.056496,75.905319,133.173157,114.639870,89.360291,190.445312,156.919006,149.610245,153.546478,119.669647,...,73.609520,226.830002,228.768921,38.453064,119.490891,349.808685,255.713577,209.325378,197.473389,40.113564
2021-01-07,97.084549,73.880081,134.515015,117.659554,93.029938,192.747116,158.108002,147.571335,158.335983,118.723045,...,73.285217,235.039993,230.410873,39.095406,119.227066,355.416992,263.817078,210.496964,198.371613,40.428280
2021-01-08,97.594048,73.223480,133.810318,117.249100,94.013748,194.114761,159.134995,148.867966,152.283493,118.840157,...,72.699654,242.460007,228.129288,39.708553,119.608147,353.820221,259.301849,212.111526,198.302521,40.877888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,155.414124,93.579231,182.020874,111.363403,136.045654,309.364075,164.773499,239.270920,171.022018,187.664612,...,115.271065,113.760002,293.922424,33.394753,89.333771,508.041840,362.484375,216.718323,206.907776,82.224838
2022-03-28,156.030655,94.323311,181.221664,112.533562,136.641235,305.893127,168.990494,241.932297,171.377625,187.092590,...,114.508064,116.169998,292.703522,33.997253,90.660606,508.210175,362.215240,219.039993,210.488876,79.918304
2022-03-29,156.232941,95.366943,178.893127,114.090469,140.393234,306.994385,169.315002,249.935745,176.711548,188.315552,...,115.271065,121.180000,297.431641,34.718292,94.611412,505.764191,364.009399,226.332382,213.672058,79.493668


In [41]:
rd1= rd.pct_change().shift(-1)
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,0.010341,-0.001349,0.000303,0.001922,0.031426,-0.005848,0.010004,-0.002894,0.002173,0.005337,...,0.009686,0.012892,-0.008242,0.016475,0.000854,-0.013448,0.025778,-0.014925,-0.004485,0.048193
2021-01-05,-0.008638,0.007982,0.043890,0.022755,0.013728,0.057377,-0.024897,0.026827,0.038232,0.036993,...,0.059243,-0.034396,0.059276,0.016465,0.043697,0.041966,0.089060,-0.008811,-0.009457,0.025517
2021-01-06,0.010703,-0.026681,0.010076,0.026341,0.041066,0.012086,0.007577,-0.013628,0.031193,-0.007910,...,-0.004406,0.036194,0.007177,0.016705,-0.002208,0.016032,0.031690,0.005597,0.004549,0.007846
2021-01-07,0.005248,-0.008887,-0.005239,-0.003488,0.010575,0.007096,0.006496,0.008786,-0.038226,0.000986,...,-0.007990,0.031569,-0.009902,0.015683,0.003196,-0.004493,-0.017115,0.007670,-0.000348,0.011121
2021-01-08,0.016314,-0.005052,-0.005772,0.002917,0.025115,0.000149,-0.021519,-0.006690,0.029282,-0.005912,...,0.000991,-0.020457,-0.007197,0.002696,-0.012581,-0.007265,-0.010378,-0.011882,-0.015082,0.030356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,0.003967,0.007951,-0.004391,0.010508,0.004378,-0.011220,0.025593,0.011123,0.002079,-0.003048,...,-0.006619,0.021185,-0.004147,0.018042,0.014853,0.000331,-0.000742,0.010713,0.017308,-0.028052
2022-03-28,0.001296,0.011064,-0.012849,0.013835,0.027459,0.003600,0.001920,0.033081,0.031124,0.006537,...,0.006663,0.043126,0.016153,0.021209,0.043578,-0.004813,0.004953,0.033292,0.015123,-0.005313
2022-03-29,0.009681,0.007397,0.004743,-0.014776,-0.039876,-0.005189,-0.017801,-0.018389,-0.011571,-0.006180,...,-0.004633,-0.022198,0.007073,0.008250,-0.030769,0.019756,-0.016457,-0.018280,0.010102,0.017118


In [42]:
#first: ranks assigned in order they appear in the array
rd2=rd1.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,33,14,15,20,40,9,32,12,21,24,...,31,35,6,37,16,5,39,3,11,43
2021-01-05,8,13,36,24,19,39,3,27,30,28,...,40,1,41,20,35,34,43,7,6,25
2021-01-06,24,2,22,35,43,27,19,3,39,8,...,9,42,18,29,11,28,40,17,16,20
2021-01-07,28,8,13,17,36,31,30,34,1,22,...,11,44,6,41,26,14,3,33,19,37
2021-01-08,38,22,21,28,41,25,5,19,42,20,...,26,7,15,27,10,14,12,11,9,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,21,16,27,4,3,28,22,39,44,24,...,37,2,36,25,5,8,35,19,7,41
2022-03-25,22,27,9,31,23,4,44,36,19,13,...,8,41,11,40,38,18,16,32,39,2
2022-03-28,7,20,2,24,35,11,9,40,38,14,...,15,42,26,32,43,5,12,41,25,4


In [43]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,4,2,2,3,5,1,4,2,3,3,...,4,4,1,5,2,1,5,1,2,5
2021-01-05,1,2,4,3,3,5,1,3,4,4,...,5,1,5,3,4,4,5,1,1,3
2021-01-06,3,1,3,4,5,3,3,1,5,1,...,1,5,2,4,2,4,5,2,2,3
2021-01-07,4,1,2,2,4,4,4,4,1,3,...,2,5,1,5,3,2,1,4,3,5
2021-01-08,5,3,3,4,5,3,1,3,5,3,...,3,1,2,3,2,2,2,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,3,2,3,1,1,4,3,5,5,3,...,5,1,4,3,1,1,4,3,1,5
2022-03-25,3,3,1,4,3,1,5,4,3,2,...,1,5,2,5,5,2,2,4,5,1
2022-03-28,1,3,1,3,4,2,1,5,5,2,...,2,5,3,4,5,1,2,5,3,1


#####Functions
***

In [44]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

#####Naive in Ranks
***
1st Method-Previous Day

In [45]:
train_set=253 # one year as "training"
naive=rd3.shift(1).copy()
# for i in naive:
#   naive[i]=(naive[i].shift(1))
naive=naive[253:]
naive

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,2.0,3.0,4.0,4.0,1.0,5.0,1.0,4.0,4.0,5.0,...,5.0,1.0,3.0,3.0,2.0,1.0,5.0,3.0,3.0,5.0
2022-01-05,5.0,5.0,4.0,2.0,1.0,4.0,2.0,3.0,4.0,3.0,...,2.0,2.0,3.0,1.0,1.0,4.0,1.0,3.0,1.0,5.0
2022-01-06,2.0,2.0,2.0,1.0,4.0,4.0,2.0,3.0,3.0,3.0,...,4.0,5.0,5.0,5.0,2.0,1.0,3.0,2.0,1.0,5.0
2022-01-07,3.0,5.0,5.0,2.0,1.0,4.0,3.0,2.0,3.0,4.0,...,4.0,1.0,5.0,4.0,1.0,2.0,1.0,2.0,1.0,4.0
2022-01-10,5.0,3.0,1.0,2.0,2.0,3.0,2.0,3.0,1.0,3.0,...,3.0,1.0,4.0,4.0,1.0,5.0,2.0,1.0,4.0,3.0
2022-01-11,2.0,1.0,2.0,4.0,4.0,5.0,5.0,2.0,4.0,3.0,...,3.0,5.0,4.0,1.0,5.0,3.0,5.0,3.0,3.0,5.0
2022-01-12,3.0,4.0,2.0,2.0,5.0,4.0,2.0,4.0,4.0,2.0,...,3.0,1.0,1.0,4.0,1.0,5.0,1.0,4.0,1.0,2.0
2022-01-13,1.0,5.0,2.0,2.0,2.0,4.0,1.0,4.0,3.0,3.0,...,5.0,1.0,5.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0
2022-01-14,5.0,4.0,3.0,2.0,5.0,4.0,4.0,3.0,2.0,1.0,...,4.0,2.0,5.0,1.0,1.0,4.0,1.0,3.0,1.0,5.0


In [46]:
import plotly.express as px
import plotly.graph_objects as go

In [48]:
fig = px.line(x=rd3['AMZN'].index[260:-25], y=rd3['AMZN'].iloc[260:-25],color=px.Constant("Πραγματικές κλάσεις"),labels=dict(x="Ημερομηνία", y="Κλάση", color="Μετοχές"),width=700, height=400)
fig.add_trace(go.Scatter(x=naive['AMZN'].index[7:-25], y=naive['AMZN'].iloc[7:-25],name="Προβλεπόμενες κλάσεις"))
fig.show()

In [ ]:
for i in naive:
  temp3=[]
  for j in range(len(naive[i].values)):
    temp = naive[i].values[j]
    # print(temp)
    temp2=[1,2,3,4,5]
    temp2=np.where(temp2!=temp,0,temp2)
    temp2=np.where(temp2==temp,1,temp2)
    # print(temp2)
    temp3.append(temp2)
  naive[i]=temp3
naive

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,"[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]",...,"[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-05,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]",...,"[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-06,"[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]",...,"[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-07,"[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]",...,"[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]"
2022-01-10,"[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]",...,"[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]"
2022-01-11,"[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]",...,"[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-12,"[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]",...,"[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]"
2022-01-13,"[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]",...,"[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]"
2022-01-14,"[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"


In [ ]:
def create_RPS(prob_final,RPS):
  d=prob_final['Date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['Date']=d.values
  RPS=RPS.set_index('Date')
  RPS.columns=assets
  return RPS

In [ ]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [ ]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('Date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['Date'][k]!=prob_final['Date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      prob=rd.loc[(rd.index==prob_final['Date'][k])]
      prob1=rd.loc[ (rd.index==prob_final['Date'][k+1])]
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['Date']==prob_final['Date'][k]] #find predictions of this day
      predic=predic.drop('Date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      prob=rd.loc[(rd.index==prob_final['Date'][k])]
      prob1=rd.loc[ (rd.index==rd.index[-1])]
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['Date']==prob_final['Date'][k]] #find predictions of this day
      predic=predic.drop('Date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)   
  print(RPS)           
  return RPS

In [ ]:
naive=naive.reset_index()
# RPS= RPS_data(naive,rd3)


In [ ]:

RPS= RPS_data(naive,rd)

      ID  Rank 1  Rank 2  Rank 3  Rank 4  Rank 5
0   ABBV       0       1       0       0       0
1    AEP       0       0       1       0       0
2    AIZ       0       0       0       1       0
3   ALLE       0       0       0       1       0
4   AMAT       1       0       0       0       0
5    AMP       0       0       0       0       1
6   AMZN       1       0       0       0       0
7    AVB       0       0       0       1       0
8    AVY       0       0       0       1       0
9    AXP       0       0       0       0       1
10   BDX       0       1       0       0       0
11  BF-B       0       1       0       0       0
12   BMY       0       0       1       0       0
13    BR       0       1       0       0       0
14  CARR       0       0       0       0       1
15   CDW       0       0       0       1       0
16  CHTR       1       0       0       0       0
17   CNC       1       0       0       0       0
18   CNP       0       1       0       0       0
19   COP       0    

In [ ]:
RPS= create_RPS(naive,RPS)
RPS
print(RPS.mean(axis=1).mean())

0.31777777777777777


#####Naive in Ranks 
***
2nd Method - Previous Month

In [ ]:
train_set=253 # one year as "training"
naive=rd3.copy()
for i in naive:
  naive[i]=(naive[i].shift(22))
naive=naive[253:]
naive

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,5.0,5.0,3.0,4.0,3.0,1.0,2.0,4.0,1.0,3.0,...,2.0,1.0,3.0,2.0,4.0,4.0,1.0,2.0,1.0,3.0
2022-01-05,4.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,5.0,5.0,...,3.0,1.0,3.0,2.0,5.0,4.0,5.0,5.0,1.0,2.0
2022-01-06,1.0,2.0,2.0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,...,3.0,5.0,3.0,2.0,2.0,3.0,5.0,4.0,4.0,3.0
2022-01-07,3.0,3.0,2.0,4.0,3.0,2.0,2.0,5.0,3.0,5.0,...,2.0,5.0,5.0,5.0,2.0,4.0,3.0,4.0,5.0,3.0
2022-01-10,5.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,3.0,...,5.0,1.0,2.0,1.0,3.0,5.0,4.0,5.0,1.0,4.0
2022-01-11,5.0,4.0,1.0,3.0,2.0,3.0,1.0,4.0,3.0,1.0,...,3.0,1.0,2.0,1.0,3.0,5.0,1.0,4.0,4.0,4.0
2022-01-12,4.0,4.0,3.0,3.0,1.0,2.0,2.0,5.0,3.0,1.0,...,1.0,2.0,3.0,4.0,1.0,4.0,1.0,2.0,4.0,1.0
2022-01-13,5.0,4.0,3.0,2.0,4.0,3.0,3.0,1.0,1.0,3.0,...,5.0,4.0,5.0,4.0,4.0,4.0,3.0,2.0,1.0,3.0
2022-01-14,5.0,5.0,1.0,3.0,5.0,5.0,5.0,4.0,3.0,1.0,...,1.0,5.0,2.0,2.0,1.0,5.0,3.0,3.0,4.0,1.0


In [ ]:
for i in naive:
  temp3=[]
  for j in range(len(naive[i].values)):
    temp = naive[i].values[j]
    # print(temp)
    temp2=[1,2,3,4,5]
    temp2=np.where(temp2!=temp,0,temp2)
    temp2=np.where(temp2==temp,1,temp2)
    # print(temp2)
    temp3.append(temp2)
  naive[i]=temp3
naive

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]",...,"[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]"
2022-01-05,"[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]",...,"[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]"
2022-01-06,"[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]",...,"[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]"
2022-01-07,"[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]",...,"[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]"
2022-01-10,"[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]",...,"[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]"
2022-01-11,"[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]",...,"[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]"
2022-01-12,"[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]",...,"[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]"
2022-01-13,"[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]",...,"[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]"
2022-01-14,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]",...,"[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]"


In [ ]:
naive=naive.reset_index()
# RPS= RPS_data(naive,rd3)

RPS= RPS_data(naive,rd)

      ID  Rank 1  Rank 2  Rank 3  Rank 4  Rank 5
0   ABBV       0       0       0       0       1
1    AEP       0       0       0       0       1
2    AIZ       0       0       1       0       0
3   ALLE       0       0       0       1       0
4   AMAT       0       0       1       0       0
5    AMP       1       0       0       0       0
6   AMZN       0       1       0       0       0
7    AVB       0       0       0       1       0
8    AVY       1       0       0       0       0
9    AXP       0       0       1       0       0
10   BDX       0       0       0       1       0
11  BF-B       0       0       0       1       0
12   BMY       0       0       0       0       1
13    BR       1       0       0       0       0
14  CARR       0       0       0       0       1
15   CDW       0       1       0       0       0
16  CHTR       0       0       0       0       1
17   CNC       0       0       1       0       0
18   CNP       0       0       0       0       1
19   COP       0    

In [ ]:
RPS= create_RPS(naive,RPS)
RPS
print(RPS.mean(axis=1).mean())

0.30948148148148147


In [ ]:
#save results
RPS.to_excel('Method_4-Naive.xlsx')

#####Naive in Percentage Returns
***
Method: rolling mean from 3 days 


In [ ]:
train_set=253 # one year as "training"
naive=rd.copy()
for i in naive:
  naive[i]=(naive[i].rolling(3).mean().shift(1))
naive=naive[252:]
naive

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,131.807444,86.610263,153.253586,131.106496,157.697703,299.157318,168.187500,248.036718,213.686707,162.382736,...,105.021357,190.143336,269.936737,33.500946,112.729055,499.210551,332.100006,216.432465,227.879405,59.186798
2022-01-04,131.826884,86.620021,153.773936,130.110891,157.350072,298.778859,168.588664,248.582204,211.612264,163.829554,...,105.292002,191.800003,268.820089,33.517464,112.679593,498.121114,330.940013,217.513387,225.341949,59.959236
2022-01-05,131.577326,86.607010,154.745478,130.388733,157.475891,304.080526,168.214498,249.850637,211.823003,166.951096,...,106.928780,191.553335,268.381297,33.600060,112.313571,493.654429,336.673340,219.035294,223.229619,61.620462
2022-01-06,131.729660,86.769666,154.893677,129.353429,156.277395,308.450857,167.427831,249.377441,210.970174,169.646520,...,107.946935,191.080002,266.852061,33.167257,110.790118,489.544271,337.583333,220.126165,218.440343,63.417431
2022-01-07,131.668081,86.955093,154.748769,128.926737,155.088842,313.367493,165.044332,249.150701,212.023860,171.339060,...,109.010201,190.190002,267.349803,33.150739,109.563438,478.772024,339.093333,219.569132,215.454519,64.962304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,156.758931,94.407778,179.064143,113.953440,134.489695,304.619324,163.982173,239.131444,166.026001,187.510681,...,115.570567,116.190002,289.531504,33.369497,89.957573,503.952576,356.589996,216.024068,208.612554,81.155429
2022-03-28,157.191182,94.929532,180.674708,112.237083,135.013743,306.980682,163.943670,240.501740,166.782959,187.162969,...,116.175921,114.820002,291.744659,33.402645,89.474431,506.505432,359.026672,216.010778,207.564570,82.253405
2022-03-29,158.353668,95.454567,182.229055,112.333359,137.083384,309.602956,165.804499,243.311340,169.724864,188.381602,...,117.191350,115.326668,293.697642,33.734118,90.076708,509.806600,362.016673,218.033325,208.878713,82.148837


In [ ]:
naive= naive.pct_change()
naive

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,0.000147,0.000113,0.003395,-0.007594,-0.002204,-0.001265,0.002385,0.002199,-0.009708,0.008910,...,0.002577,0.008713,-0.004137,0.000493,-0.000439,-0.002182,-0.003493,0.004994,-0.011135,0.013051
2022-01-05,-0.001893,-0.000150,0.006318,0.002135,0.000800,0.017744,-0.002219,0.005103,0.000996,0.019054,...,0.015545,-0.001286,-0.001632,0.002464,-0.003248,-0.008967,0.017324,0.006997,-0.009374,0.027706
2022-01-06,0.001158,0.001878,0.000958,-0.007940,-0.007611,0.014372,-0.004677,-0.001894,-0.004026,0.016145,...,0.009522,-0.002471,-0.005698,-0.012881,-0.013564,-0.008326,0.002703,0.004980,-0.021454,0.029162
2022-01-07,-0.000467,0.002137,-0.000936,-0.003299,-0.007605,0.015940,-0.014236,-0.000909,0.004994,0.009977,...,0.009850,-0.004658,0.001865,-0.000498,-0.011072,-0.022005,0.004473,-0.002531,-0.013669,0.024360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,0.000481,0.006120,0.008738,-0.013394,0.010945,0.014151,0.004406,-0.002235,-0.007803,-0.000353,...,0.008721,0.004033,0.009299,-0.002872,-0.002385,0.003825,0.010561,0.000415,-0.003306,0.004774
2022-03-28,0.002757,0.005527,0.008994,-0.015062,0.003897,0.007752,-0.000235,0.005730,0.004559,-0.001854,...,0.005238,-0.011791,0.007644,0.000993,-0.005371,0.005066,0.006833,-0.000062,-0.005024,0.013529
2022-03-29,0.007395,0.005531,0.008603,0.000858,0.015329,0.008542,0.011350,0.011682,0.017639,0.006511,...,0.008740,0.004413,0.006694,0.009924,0.006731,0.006518,0.008328,0.009363,0.006331,-0.001271


In [ ]:
#I will create their rankings 
naive=naive.rank(axis=1,method='first')
naive=naive.dropna()
naive=naive[:-1]
naive.astype(int)
naive2=naive.copy()
naive2.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,28,27,35,5,21,24,32,31,4,44,...,33,43,11,29,25,23,16,40,2,45
2022-01-05,20,23,33,29,25,41,17,32,26,42,...,38,22,21,31,15,7,39,35,6,45
2022-01-06,31,33,30,16,17,42,21,27,22,44,...,38,24,19,10,9,15,35,36,4,45
2022-01-07,25,32,22,19,14,43,8,23,35,41,...,40,17,30,24,11,3,33,20,9,45
2022-01-10,29,42,32,15,11,31,17,20,24,33,...,37,22,41,26,1,4,5,19,2,44
2022-01-11,26,31,25,9,13,32,17,21,12,34,...,40,16,41,39,2,4,5,6,7,38
2022-01-12,32,27,28,20,8,38,33,17,16,35,...,37,19,40,24,10,23,21,9,14,41
2022-01-13,32,12,7,28,45,37,31,26,15,18,...,27,14,30,24,22,41,33,11,19,40
2022-01-14,8,21,16,27,45,40,18,28,35,15,...,33,9,32,10,31,22,44,30,6,41


In [ ]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
naive3=naive2.copy()
naive3=rank_data(naive2)
naive3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,4,3,4,1,3,3,4,4,1,5,...,4,5,2,4,3,3,2,5,1,5
2022-01-05,3,3,4,4,3,5,2,4,3,5,...,5,3,3,4,2,1,5,4,1,5
2022-01-06,4,4,4,2,2,5,3,3,3,5,...,5,3,3,2,1,2,4,4,1,5
2022-01-07,3,4,3,3,2,5,1,3,4,5,...,5,2,4,3,2,1,4,3,1,5
2022-01-10,4,5,4,2,2,4,2,3,3,4,...,5,3,5,3,1,1,1,3,1,5
2022-01-11,3,4,3,1,2,4,2,3,2,4,...,5,2,5,5,1,1,1,1,1,5
2022-01-12,4,3,4,3,1,5,4,2,2,4,...,5,3,5,3,2,3,3,1,2,5
2022-01-13,4,2,1,4,5,5,4,3,2,2,...,3,2,4,3,3,5,4,2,3,5
2022-01-14,1,3,2,3,5,5,2,4,4,2,...,4,1,4,2,4,3,5,4,1,5


In [ ]:
for i in naive3:
  temp3=[]
  for j in range(len(naive3[i].values)):
    temp = naive3[i].values[j]
    # print(temp)
    temp2=[1,2,3,4,5]
    temp2=np.where(temp2!=temp,0,temp2)
    temp2=np.where(temp2==temp,1,temp2)
    # print(temp2)
    temp3.append(temp2)
  naive3[i]=temp3
naive3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,"[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]",...,"[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-05,"[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]",...,"[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-06,"[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]",...,"[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-07,"[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]",...,"[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-10,"[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]",...,"[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-11,"[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]",...,"[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-12,"[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]",...,"[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-13,"[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 1, 0, 0, 0]",...,"[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]"
2022-01-14,"[1, 0, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 0, 1]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]",...,"[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]","[0, 0, 1, 0, 0]","[0, 0, 0, 0, 1]","[0, 0, 0, 1, 0]","[1, 0, 0, 0, 0]","[0, 0, 0, 0, 1]"


In [ ]:
naive3=naive3.reset_index()
# RPS= RPS_data(naive3,rd3)

RPS= RPS_data(naive3,rd)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
13    BR -0.040835       1.0  1.0   1.0   0.0   0.0   0.0   0.0
21   CZR -0.036271       2.0  1.0   1.0   0.0   0.0   0.0   0.0
36  PYPL -0.033066       3.0  1.0   1.0   0.0   0.0   0.0   0.0
22    DG -0.024235       4.0  1.0   1.0   0.0   0.0   0.0   0.0
6   AMZN -0.024170       5.0  1.0   1.0   0.0   0.0   0.0   0.0
43  VRSK -0.020637       6.0  1.0   1.0   0.0   0.0   0.0   0.0
32  META -0.020344       7.0  1.0   1.0   0.0   0.0   0.0   0.0
27  GOOG -0.017769       8.0  1.0   1.0   0.0   0.0   0.0   0.0
0   ABBV -0.017368       9.0  1.0   1.0   0.0   0.0   0.0   0.0
15   CDW -0.016970      10.0  2.0   0.0   1.0   0.0   0.0   0.0
14  CARR -0.015094      11.0  2.0   0.0   1.0   0.0   0.0   0.0
4   AMAT -0.014854      12.0  2.0   0.0   1.0   0.0   0.0   0.0
40   UNH -0.013278      13.0  2.0   0.0   1.0   0.0   0.0   0.0
3   ALLE -0.011444      14.0  2.0   0.0   1.0   0.0   0.0   0.0
23   DPZ -0.011276      15.0  2.0   0.0   1.0   0

In [ ]:
RPS= create_RPS(naive3,RPS)
RPS
print(RPS.mean(axis=1).mean())

0.30977777777777776
